In [9]:
import pandas as pd
import pandas_gbq
from pandas.io import gbq
from google.cloud import bigquery
from google.oauth2 import service_account
import sqlite3

In [10]:
# These first two values will be different on your machine. 
service_path = "/Users/liamhettinger/Documents/Fall_MSBA/Applied_Data_Analytics/wedge/"
service_file = 'fleet-geode-327423-3bd2c958746e.json' # change this to your authentication information  
gbq_proj_id = 'fleet-geode-327423' # change this to your poroject. 
gbq_dataset_id = 'Wedge'

# And this should stay the same. 
private_key = service_path + service_file

In [11]:
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [12]:
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [15]:
table_ref = client.create_table(table = 'fleet-geode-327423.Wedge.SalesByHour')

In [16]:
sql1 ="""
SELECT
  EXTRACT(date FROM datetime) AS day,
  EXTRACT(hour FROM datetime) AS hour,
  SUM(total) AS sales,
  count(distinct(date(datetime) || register_no || emp_no || trans_no)) as transactions,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') 
  THEN 
  -1 
  ELSE 
  1 
  END) as item_quantity
FROM
  `fleet-geode-327423.Wedge.transArchive*`
WHERE department != 0 and
      department != 15 and
      trans_status != 'M' and
      trans_status != 'C' and
      trans_status != 'J' and
      (trans_status = '' or 
      trans_status = ' ' or 
      trans_status = 'V' or 
      trans_status = 'R')
GROUP BY
  day, hour
ORDER BY
  day, hour
"""

In [17]:
SalesByHour = pandas_gbq.read_gbq(sql1, project_id=gbq_proj_id)

Downloading: 100%|██████████| 39097/39097 [00:02<00:00, 13163.36rows/s]


In [18]:
table_ref = client.create_table(table = 'fleet-geode-327423.Wedge.Salesbyownerbyyearbymonth')

In [19]:
sql2 = """SELECT
  card_no,
  EXTRACT(MONTH from datetime) AS month,
  EXTRACT(YEAR from datetime) AS year,
  SUM(total) AS sales,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS transactions,
  SUM(CASE WHEN (trans_status = 'V' or trans_status = 'R') 
  THEN 
  -1 
  ELSE 
  1 
  END) as item_quantity
FROM
  `fleet-geode-327423.Wedge.transArchive_*`
WHERE
  department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY
  card_no,
  month,
  year
ORDER BY
  card_no,
  month,
  year"""

In [20]:
Salesbyownerbyyearbymonth = pandas_gbq.read_gbq(sql2, project_id=gbq_proj_id)

Downloading: 100%|██████████| 783240/783240 [00:31<00:00, 25258.98rows/s]


Manually created the department look-up page

In [ ]:
sql3 = """
SELECT
  upc,
  description,
  trans.department,
  dept.department_name,
  EXTRACT(MONTH from datetime) AS month,
  EXTRACT(YEAR from datetime) AS year,
  SUM(total) AS sales,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS transactions,
  SUM(CASE WHEN (trans_status = 'V' or trans_status = 'R') 
  THEN 
  -1 
  ELSE 
  1 
  END) as item_quantity
  FROM
      `fleet-geode-327423.Wedge.transArchive_*` as trans 
  LEFT JOIN
      fleet-geode-327423.Wedge.Department_lookup as dept on trans.department = dept.department
  WHERE
  trans.department != 0
  AND trans.department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
  GROUP BY
      upc,
      description,
      department,
      year,
      month
  ORDER BY
      year,
      month
  """

In [ ]:
SalesByProductDescriptionByYearByMonth = pandas_gbq.read_gbq(sql2, project_id=gbq_proj_id)

In [21]:
import sqlite3

In [22]:
database = sqlite3.connect("Wedge_SQL_DB.db")
SalesByHour.to_sql("SalesByHour", database, if_exists="replace")
Salesbyownerbyyearbymonth.to_sql("SalesByOwnerByYearByMonth", database, if_exists="replace")
#SalesByProductDescriptionByYearByMonth.to_sql("SalesByProductDescriptionByYearByMonth", database, if_exists="replace")

In [23]:
database.close()

References:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_gbq.html

https://pandas.pydata.org/docs/reference/api/pandas.read_gbq.html